# Data cleaning

In [1]:
# External modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## ACT data

In [2]:
# Read in data for State, Participation rate, Composite score
act_17 = pd.read_csv('../data/Raw-data/act_2017.csv')[['State','Participation','Composite']] # Select columns consistent with '18 and '19 data
act_18 = pd.read_csv('../data/Raw-data/act_2018.csv')
act_19 = pd.read_csv('../data/Raw-data/act_2019.csv')

In [3]:
# Check that all datasets have the same number of rows
print(len(act_17))
print(len(act_18))
print(len(act_19))

52
52
52


In [4]:
act_17.head()

,State,Participation,Composite
0,National,60%,21.0
1,Alabama,100%,19.2
2,Alaska,65%,19.8
3,Arizona,62%,19.7
4,Arkansas,100%,19.4


In [5]:
act_18.head()

,State,Participation,Composite
0,Alabama,100%,19.1
1,Alaska,33%,20.8
2,Arizona,66%,19.2
3,Arkansas,100%,19.4
4,California,27%,22.7


In [6]:
act_19.head()

,State,Participation,Composite
0,Alabama,100%,18.9
1,Alaska,38%,20.1
2,Arizona,73%,19.0
3,Arkansas,100%,19.3
4,California,23%,22.6


In [7]:
act_17.dtypes

State            object
Participation    object
Composite        object
dtype: object

In [8]:
# Format column names to lowercase with no space
format_names = lambda x: x.lower().replace(' ', '_')

act_17.rename(columns = format_names, inplace = True)
act_18.rename(columns = format_names, inplace = True)
act_19.rename(columns = format_names, inplace = True)

In [9]:
# Remove '%' and change type to float
def make_decimal(string_percent):
    string_percent = string_percent.replace('%','')
    try:
        return float(string_percent) / 100
    except:
        return np.nan

# Change participation column to decimal float
act_17['participation'] = act_17['participation'].apply(make_decimal)
act_18['participation'] = act_18['participation'].apply(make_decimal)
act_19['participation'] = act_19['participation'].apply(make_decimal)

In [10]:
# Correcting data entry error
data_entry = act_17['composite'] == '20.2x'
act_17.loc[data_entry, 'composite'] = '20.2'

# Change type of composite column to float
act_17['composite'] = act_17['composite'].astype(float)

In [11]:
# Verify
act_17.head()

,state,participation,composite
0,National,0.60,21.0
1,Alabama,1.00,19.2
2,Alaska,0.65,19.8
3,Arizona,0.62,19.7
4,Arkansas,1.00,19.4


In [12]:
# Correct spelling
act_18.loc[act_18['state'] == 'District of columbia', 'state'] = 'District of Columbia'

In [13]:
# Drop duplicate 'Maine' row
act_18.drop_duplicates(inplace = True)

In [14]:
# Drop national column since it is missing from 2018 data
act_17.drop([0], inplace = True)
act_19.drop([51], inplace = True)

In [15]:
# Checking for scores outside of valid range
invalid_score = lambda x: (x.loc[:,'composite'] < 2) | (x.loc[:,'composite'] > 36)

inv_17 = invalid_score(act_17)
inv_18 = invalid_score(act_18)
inv_19 = invalid_score(act_19)

In [16]:
# All scores are valid
print(inv_17.unique())
print(inv_18.unique())
print(inv_19.unique())

[False]
[False]
[False]


In [17]:
# Add year column before merging
act_17['year'] = 2017
act_18['year'] = 2018
act_19['year'] = 2019

In [18]:
# Combine all years into one dataframe
act = pd.concat([act_17, act_18, act_19], ignore_index = True)
act.shape

(153, 4)

In [19]:
# Save to local csv file
act.to_csv('../Data/act_all.csv', index = False)

## SAT data

In [20]:
# Read in SAT data
sat_17 = pd.read_csv('../Data/Raw-data/sat_2017.csv')
sat_18 = pd.read_csv('../Data/Raw-data/sat_2018.csv')
sat_19 = pd.read_csv('../Data/Raw-data/sat_2019.csv')

In [21]:
# Check that all datasets have the same number of rows
print(len(sat_17))
print(len(sat_18))
print(len(sat_19))

51
51
53


In [22]:
sat_17.head()

,State,Participation,Evidence-Based Reading and Writing,Math,Total
0,Alabama,5%,593,572,1165
1,Alaska,38%,547,533,1080
2,Arizona,30%,563,553,1116
3,Arkansas,3%,614,594,1208
4,California,53%,531,524,1055


In [23]:
sat_18.head()

,State,Participation,Evidence-Based Reading and Writing,Math,Total
0,Alabama,6%,595,571,1166
1,Alaska,43%,562,544,1106
2,Arizona,29%,577,572,1149
3,Arkansas,5%,592,576,1169
4,California,60%,540,536,1076


In [24]:
sat_19.head()

,State,Participation Rate,EBRW,Math,Total
0,Alabama,7%,583,560,1143
1,Alaska,41%,556,541,1097
2,Arizona,31%,569,565,1134
3,Arkansas,6%,582,559,1141
4,California,63%,534,531,1065


In [25]:
# Set the same column names across all 3 years
col_names = ['state','participation','ebrw','math','total']

sat_17.columns = col_names
sat_18.columns = col_names
sat_19.columns = col_names

In [26]:
# Change participation column to decimal float
sat_17['participation'] = sat_17['participation'].apply(make_decimal)
sat_18['participation'] = sat_18['participation'].apply(make_decimal)
sat_19['participation'] = sat_19['participation'].apply(make_decimal)

In [27]:
# Identify rows unique to 2019 data
for state in sat_19['state'].unique():
    if state not in sat_17['state'].unique():
        print(state)

Puerto Rico
Virgin Islands


In [28]:
# Drop Puerto Rico and Virgin Islands
pr = sat_19['state'] == 'Puerto Rico'
vi = sat_19['state'] == 'Virgin Islands'
idx_to_drop = sat_19.loc[pr|vi, :].index

sat_19.drop(index = idx_to_drop, inplace = True)

In [29]:
# Checking for scores outside of valid range
invalid_score = lambda x: (x.loc[:,'total'] < 400) | (x.loc[:,'total'] > 1600)

inv_17 = invalid_score(sat_17)
inv_18 = invalid_score(sat_18)
inv_19 = invalid_score(sat_19)

In [30]:
# All scores are valid
print(inv_17.unique())
print(inv_18.unique())
print(inv_19.unique())

[False]
[False]
[False]


In [31]:
# Add year column before merging
sat_17['year'] = 2017
sat_18['year'] = 2018
sat_19['year'] = 2019

In [32]:
# Combine all years into one dataframe
sat = pd.concat([sat_17, sat_18, sat_19], ignore_index = True)
sat.shape

(153, 6)

In [33]:
# Save to local csv file
sat.to_csv('../data/sat_all.csv', index = False)